# Numpy/Vision Workshop

The purpose of this workshop is to provide some high level description of the vision components showcasing where numpy can be of assistance.

## Numpy Basics


In [ ]:
# An image in numpy is just a multidimensional array (what one calls a tensor )
# an example of one could be :
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 8 by 8 image with 3 color channels as a numpy tensor
A = np.arange(64*3).reshape(8,8, 3)
plt.imshow(A)
plt.show()

In [ ]:
# Lets have a look at something more familiar matrices and matrix mult !

A = np.arange(9).reshape(3,3).astype(float)
print "Matrix A: \n", A, "\n"

I = np.identity(3)

print "Matrix I: \n", I, "\n"

# Matrix mult is carried out with the dot function

print "Matrix AI: \n", A.dot(I), "\n"
print "Matrix A^{2}: \n", A.dot(A), "\n"

# Element wise multiplication ( hadamard product):

print "Matrix A * I: \n", A * I, "\n"

# Likewise division
print "Matrix \\frac{A} {(I+1)}: \n", A / (I+1), "\n"

# Tranpose:

print "A^{T}: \n", A.T, "\n"

# apply a function elementwise

print "exp(A): \n", np.exp(A)

## Dealing with Shadows

Whilst image segmentation and filtering is more of a logical starting point in terms of starting simple with numpy RGB normalization is a good place to start.


### RGB Normalization

Is the following mapping:

\begin{equation}
(R',G', B') := \left(\frac{R}{(R+G+B)},\frac{G}{(R+G+B)}, \frac{B}{(R+G+B)} \right) 
\end{equation}

Where $(R',G', B')$ are the normalized pixel color channels.

In [ ]:
from sklearn.datasets import load_sample_image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


china = load_sample_image('china.jpg')   
print "China is a 3rd order tensor (multidimensional array): ", china.shape

In [ ]:
# Display the picture (useful for debuggin vision components by examining frames)
plt.imshow(china)
plt.show()

In [ ]:
# To normalize the image the temptation would be to loo through every pixel
# and apply the formula given above nonetheless one can do beter

# Sum along the 3rd axis of the tensor
# That is for every pixel add up its color channels
normalizing_factors = np.sum(china, axis=2)
# avoid division by zero
normalizing_factors[normalizing_factors == 0] = 1
h, w = normalizing_factors.shape
print "new shape: ", normalizing_factors.shape

# Now we want to divide elementwise (which numpy allows us to do quite nicely)
# But in order for broadcasting to work we need the normalizing_factor tensor3
# to have the same order as the china tensor

norm_china = china.astype(float) / normalizing_factors.reshape(h, w, 1)
plt.imshow(norm_china)
plt.show()

# Worth to look at normed pitch images to examime potential. Atm the standard
# normalization that takes place is in HSV space normalizing the color chanel
# Which is not quite the same thing (usefull for image segmentation which I 
# might cover if I have time)

## Image Processing

This bit provides more adavance numpy usage just for showcasing since opencv is the most optimal way of carrying these out. Most of the image processing that will be useful for SDP is that of applying digital filters (bluring the image, extracting edges , etc).

The arithmetic operation which one aplies a filter to an image is called convolution:

![](http://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif)

Where the yellow moving mask is the filter normally reffered to as kernel in computer vision.  The mathematical formulation of convolution for image $I$ and kernel $K$ is the following:

\begin{equation}
I_{ouput}(x,y) = \sum_{i=-\infty}^{\infty} \sum_{j=-\infty}^{\infty} I(x -i , y - j)*k(x,y)
\end{equation}

In [ ]:
import numpy
import numpy as np
from numpy.lib import stride_tricks as st


"""
These are two examples of vectorizing 
convolution. Both slower than using opencv.
Yet still a rich example of how much vectorizing
can one do and when it brings an advantage.

As an exercise one can add padding to both these functions
such that the size of the image is conserved.
"""


def partial_conv(I,k):
    """
    Partially vectorized convolution (2 loops out of 4 removed)
    """
    I_x = I.shape[0]
    I_y = I.shape[1]

    k_x = k.shape[0]
    k_y = k.shape[1]

    out = np.zeros((I_x - k_x + 1, I_y - k_y + 1))
    o_x = out.shape[0]
    o_y = out.shape[1]
    
    # loops over small things are ok (y)
    # Image kernels tend to be small
    for y in xrange(k_y):
        for x in xrange(k_x):
            out += k[y,x] * I[y:o_y+y, x:o_x+x]
    return out


def vectorized_conv(I, k, stridz=(1,1)):
    """
    Vectorized convolution using numpy strides
    Faster for small dim images than the partial vectorization
    """
    h, w = I.shape
    k_y, k_x = k.shape
    ystep , xstep = stridz
    nh, nw = ((h - k_y + 1) / ystep, (w - k_x + 1) / xstep)

    # This stride bassically generates a permutation
    # matrix which allows to do the convolution operation in a fully
    # vectorized form. This also allows automatic striding.
    windows = st.as_strided(I,
                            (nh, nw, k_y, k_x),
                            (I.strides[-2] * ystep,
                             I.strides[-1] * xstep,
                             I.strides[-2],
                             I.strides[-1]))
    
    # flattening along final dimension allows us to express
    # convolution as a matrix mult opp
    # flattening along kernel strides
    woo = windows.reshape(nh, nw, k_y* k_x)
    # flattening kerenel
    koo = k.ravel()
    
    # convolution as matrix multiplication
    # equivalent mults: 
    # dt = np.dot(woo, koo)
    # dt = woo.dot(koo)
    # O(n^{2.3}) usual bound on fast matrix mult
    # using strassen like algorithms
    # bottleneck for big images lies partly in this line
    dt= np.einsum('ijk, k -> ij', woo, koo)
    return dt

In [ ]:
from scipy.misc import lena
import matplotlib.pyplot as plt

# Example image:

l = lena()
# Edge detection kernels/masks/filters:
sobel = np.array([[1, 0, -1],
                  [2, 0, -2],
                  [1, 0, -1]])

laplacian_of_gaussian = np.array([[0, 0, 1, 0, 0],
                                  [0, 1, 2, 1, 0],
                                  [1, 2, -16, 2, 1],
                                  [0, 1, 2, 1, 0],
                                  [0, 0, 1, 0, 0] ])


#Bluring using a uniform filter
uniform = (1 / 25.0)*np.ones((10,10))

# Convolutions
edge1 = vectorized_conv(l, laplacian_of_gaussian)
edge2 = vectorized_conv(l, sobel)
blur1 = vectorized_conv(l, uniform)

plt.gray()
plt.imshow(edge1)
plt.show()
plt.gray()
plt.imshow(edge2)
plt.show()
plt.gray()
plt.imshow(blur1)
plt.show()

In [ ]:
# But you wan to be doing this kind of thing with open cv
# Since specific high usage filters have their own custom
# optimizations:
import cv2 as cv


uniform = (1 / 100.0)*np.ones((10,10))
img = np.ones((50,50))

# Time benchmarks:
%timeit cv.blur(img,(10,10))
%timeit cv.filter2D(img,-1,uniform)
%timeit vectorized_conv(img, uniform)
%timeit partial_conv(img, uniform)

# The point of learning how to vectorize is that there are things opencv is
# missing like the rgb normalization shown earlier on

## Segmentation (Hand Calibrated by Default)

The segementation covered here will be simple image threshholding. The idea is for a given color channel pick an upper and a lower bound for which your object is in  intensity wise and set everything outside that bound to  0.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

# Ouput binary image (will encode the thresholding)
bin_out = np.ones((china.shape[0], china.shape[1]))


# Color for red channel used to find thresholds
flat_red_nc = norm_china[:,:,0].ravel()
plt.hist(flat_red_nc, 1000)
plt.show()


# Upon inspecting the histogram  (note this can be done algorithmically)
# This is quite a complex image so the histogram method is not
# the most effective
blurr = True
if blurr:
    norm_china2 =  cv.blur(norm_china,(5,5))
else:
    norm_china2 = norm_china

bin_out[norm_china2[:,:,0] < 0.45 ] = 0
bin_out[norm_china2[:,:,0] > 0.59 ] = 0
plt.gray()
plt.imshow(bin_out)
plt.show()


In [ ]:
# Lets try and find the missing tiles by
# Taking a disjunction with the blue channel

flat_blue_nc = norm_china[:,:,2].ravel()
plt.hist(flat_blue_nc, 1000)
plt.show()

In [ ]:
bin_out_b = np.ones((china.shape[0], china.shape[1]))
bin_out_b[norm_china[:,:,2] < 0.38 ] = 0


# Morpholocial transform open
# Removes noise and inflates week segements (erosion + dilation)
open_kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(2,2))
bin_out_bo = cv.morphologyEx(bin_out_b, cv.MORPH_OPEN, open_kernel)

plt.gray()
plt.imshow(bin_out_b.astype(int))
plt.show()

plt.gray()
plt.imshow(bin_out_bo.astype(int))
plt.show()


plt.gray()
plt.imshow(bin_out_bo.astype(int) | bin_out.astype(int))
plt.show()